### Imports

In [52]:
import pandas as pd

## Preparing data

### Loading dataset

In [64]:
aggregated_dataset = pd.read_csv('per_day_participant_dataset_co.csv')

In [65]:
aggregated_dataset.tail()

,Unnamed: 0,mood,mood_count,circumplex.arousal,circumplex.arousal_count,circumplex.valence,circumplex.valence_count,activity,activity_count,screen,...,appCat.social,appCat.social_count,appCat.travel,appCat.travel_count,appCat.unknown,appCat.unknown_count,appCat.utilities,appCat.utilities_count,appCat.weather,appCat.weather_count
2149,"('AS14.33', Timestamp('2014-05-27 00:00:00'))",31.0,5,-3.0,5,2.0,5,0.304902,24,4089.879001,...,2010.364,40,0.000,0,0.000,0,56.173,3,0.0,0
2150,"('AS14.33', Timestamp('2014-05-28 00:00:00'))",41.0,5,0.0,5,6.0,5,2.479228,24,11411.693819,...,5361.211,54,0.000,0,0.000,0,30.666,6,0.0,0
2151,"('AS14.33', Timestamp('2014-05-29 00:00:00'))",21.0,3,-4.0,3,3.0,3,4.064507,24,3569.341000,...,1789.922,9,0.939,1,0.000,0,3.199,2,0.0,0
2152,"('AS14.33', Timestamp('2014-05-30 00:00:00'))",34.0,5,-4.0,5,-2.0,5,4.050925,21,9497.646999,...,3166.409,42,1052.648,28,8.072,3,232.825,15,0.0,0
2153,"('AS14.33', Timestamp('2014-05-31 00:00:00'))",7.0,1,-2.0,1,1.0,1,0.000000,0,0.000000,...,0.000,0,0.000,0,0.000,0,0.000,0,0.0,0


In [67]:
from pandas import Timestamp


aggregated_dataset["date_time"]= pd.to_datetime([eval(string)[1] for string in aggregated_dataset['Unnamed: 0']])
aggregated_dataset["participant_id"]= [eval(string)[0] for string in aggregated_dataset['Unnamed: 0']]


In [68]:
aggregated_dataset.drop(columns=['Unnamed: 0'], inplace=True)

In [69]:
# reorder columns
aggregated_dataset = aggregated_dataset[aggregated_dataset.columns.to_list()[-2:] + aggregated_dataset.columns.to_list()[:-2]]

In [70]:
aggregated_dataset.head()

,date_time,participant_id,mood,mood_count,circumplex.arousal,circumplex.arousal_count,circumplex.valence,circumplex.valence_count,activity,activity_count,...,appCat.social,appCat.social_count,appCat.travel,appCat.travel_count,appCat.unknown,appCat.unknown_count,appCat.utilities,appCat.utilities_count,appCat.weather,appCat.weather_count
0,2014-02-17,AS14.01,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1,2014-02-18,AS14.01,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2,2014-02-19,AS14.01,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
3,2014-02-20,AS14.01,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
4,2014-02-21,AS14.01,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


### Counting missing values

In [91]:
# count the number of rows where mood_count is 0
zero_mood_count = aggregated_dataset[aggregated_dataset['mood_count'] == 0].shape[0]
print('Number of rows where mood_count is 0:', zero_mood_count, 'out of', aggregated_dataset.shape[0])

Number of rows where mood_count is 0: 886 out of 2154


In [92]:
counts_of_empty_mood = aggregated_dataset[aggregated_dataset['mood_count'] == 0].groupby('participant_id').size()
counts_of_empty_mood


participant_id
AS14.01    31
AS14.02    27
AS14.03    34
AS14.05    24
AS14.06    36
AS14.07     2
AS14.08    16
AS14.09    32
AS14.12    35
AS14.13    24
AS14.14    34
AS14.15    26
AS14.16    25
AS14.17    22
AS14.19    32
AS14.20    31
AS14.23    38
AS14.24     7
AS14.25    51
AS14.26    35
AS14.27    46
AS14.28    43
AS14.29    46
AS14.30    31
AS14.31    47
AS14.32    50
AS14.33    61
dtype: int64

In [73]:
partecipants = aggregated_dataset['participant_id'].unique()
current_participant = partecipants[0]
# count lenght of empty mood sequences
empty_mood_sequences = {}
"""
Dictionry with the form of:
{
    'participant_id': {
        'length_sequence': count
    }
}

Example:

{
    'AS14.01': {
        1: 10,
        2: 5
    },
    'AS14.02': {
        1: 7,
        2: 3
    }
}
"""


for participant in partecipants:
    current_length = 0
    for index, row in aggregated_dataset[aggregated_dataset['participant_id'] == participant].iterrows():
        if row['mood_count'] == 0:
            current_length += 1
            if index == len(aggregated_dataset[aggregated_dataset['participant_id'] == participant]) - 1:
                
                if participant not in empty_mood_sequences:
                    empty_mood_sequences[participant] = {}
                if current_length not in empty_mood_sequences[participant]:
                    empty_mood_sequences[participant][current_length] = 0
                empty_mood_sequences[participant][current_length] += 1

                current_length = 0
        else:
            if current_length != 0:
                if participant not in empty_mood_sequences:
                    empty_mood_sequences[participant] = {}
                if current_length not in empty_mood_sequences[participant]:
                    empty_mood_sequences[participant][current_length] = 0
                empty_mood_sequences[participant][current_length] += 1
            current_length = 0

empty_mood_sequences

{'AS14.01': {9: 1, 21: 1, 1: 1},
 'AS14.02': {27: 1},
 'AS14.03': {32: 1, 1: 1},
 'AS14.05': {24: 1},
 'AS14.06': {35: 1, 1: 1},
 'AS14.07': {2: 1},
 'AS14.08': {16: 1},
 'AS14.09': {32: 1},
 'AS14.12': {24: 1, 11: 1},
 'AS14.13': {24: 1},
 'AS14.14': {31: 1, 1: 1, 2: 1},
 'AS14.15': {25: 1, 1: 1},
 'AS14.16': {24: 1, 1: 1},
 'AS14.17': {15: 1, 6: 1, 1: 1},
 'AS14.19': {32: 1},
 'AS14.20': {31: 1},
 'AS14.23': {33: 1, 1: 1},
 'AS14.24': {5: 1, 1: 1},
 'AS14.25': {50: 1, 1: 1},
 'AS14.26': {34: 1, 1: 1},
 'AS14.27': {45: 1, 1: 1},
 'AS14.28': {42: 1, 1: 1},
 'AS14.29': {44: 1, 1: 2},
 'AS14.30': {31: 1},
 'AS14.31': {44: 1, 1: 1},
 'AS14.32': {43: 1, 2: 1, 1: 3},
 'AS14.33': {58: 1, 2: 1, 1: 1}}

In [96]:
# plot mood_count per participant
import matplotlib.pyplot as plt
import os

if not os.path.exists('participant_mood_count'):
    os.makedirs('participant_mood_count')

for participant in partecipants:
    plt.plot(aggregated_dataset[aggregated_dataset['participant_id'] == participant]['date_time'], aggregated_dataset[aggregated_dataset['participant_id'] == participant]['mood_count'])
    plt.title('Participant ' + participant + ' mood count')
    plt.savefig('participant_mood_count/' + participant + '_mood_count.png')
    plt.close()

In [ ]:
# plot per partecipant the sum of the variables that end with _count
for participant in partecipants:
    plt.plot(aggregated_dataset[aggregated_dataset['participant_id'] == participant]['date_time'], aggregated_dataset[aggregated_dataset['participant_id'] == participant].iloc[:, 3::2].sum(axis=1))
    plt.title('Participant ' + participant + ' sum of the variables that end with _count')
    plt.show()




In [ ]:
#Plot both
def plot_counts_per_participant(df):
    partecipants = df['participant_id'].unique()
    for participant in partecipants:
        plt.plot(df[df['participant_id'] == participant]['date_time'], df[df['participant_id'] == participant]['mood_count'])
        plt.title('Participant ' + participant + ' mood count')
        plt.show()
        plt.plot(df[df['participant_id'] == participant]['date_time'], df[df['participant_id'] == participant].iloc[:, 3::2].sum(axis=1))
        plt.title('Participant ' + participant + ' sum of the _count variables')
        plt.show()


plot_counts_per_participant(aggregated_dataset)

## Cleaning

In [88]:
# strip the first rows and the last rows with mood_count = 0 per participant
aggregated_dataset_cleaned = pd.DataFrame()
for participant in partecipants:
    participant_data = aggregated_dataset[aggregated_dataset['participant_id'] == participant]
    participant_data.reset_index(drop=True, inplace=True)
    first_non_zero_mood = participant_data[participant_data['mood_count'] != 0].index[0]
    last_non_zero_mood = participant_data[participant_data['mood_count'] != 0].index[-1]
    aggregated_dataset_cleaned = pd.concat([aggregated_dataset_cleaned, participant_data[first_non_zero_mood:last_non_zero_mood+1]])

In [89]:
print('Original dataset shape:', aggregated_dataset.shape)
print('Cleaned dataset shape:', aggregated_dataset_cleaned.shape)

Original dataset shape: (2154, 40)
Cleaned dataset shape: (1331, 40)
